In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os


## coverthe spreadsheet into csv fiels

In [ ]:
import pandas as pd

# Define the file path
file_path = r'C:\Users\Hazem Hamdy\Desktop\mobadra\voda.xlsx'

# Load the Excel file
excel_data = pd.ExcelFile(file_path)

# Get the names of the sheets in the Excel file
sheet_names = excel_data.sheet_names

# Convert each sheet to a CSV file
for sheet_name in sheet_names:
    # Read the sheet into a DataFrame
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # Define the CSV file path
    csv_file_path = f'C:\\Users\\Hazem Hamdy\\Desktop\\mobadra\\csc_voda\\{sheet_name}.csv'
    
    # Save the DataFrame to a CSV file
    df.to_csv(csv_file_path, index=False)
    print(f"Saved {sheet_name} to {csv_file_path}")

## Concat the data and add the locatoin column

In [ ]:
# Define the directory containing the CSV files
directory = r'C:\Users\Hazem Hamdy\Desktop\mobadra\csc_voda'

# Use glob to find all CSV files in the directory
file_paths = glob.glob(os.path.join(directory, '*.csv'))

# Create an empty DataFrame to hold the combined data
combined_df1 = pd.DataFrame()

# Process each file
for file_path in file_paths:
    # Extract the location name from the file name
    location = os.path.basename(file_path).split('.')[0]
    
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Ensure 'DIM' and 'Task' columns exist
    if 'DIM' not in df.columns or 'Task' not in df.columns:
        continue
    
    # Add 'Location' column with the extracted location name
    df['Location'] = location
    
    # Reorder columns to have 'Location' at the beginning
    df = df[['Location', 'DIM', 'Task', 'APPROVAL'] + [col for col in df.columns if col not in ['Location', 'DIM', 'Task', 'APPROVAL']]]
    
    # Append to combined DataFrame
    combined_df1 = pd.concat([combined_df1, df], ignore_index=True)

# Save the combined DataFrame to a new CSV file
# combined_df1.to_csv(r'C:\Users\Hazem Hamdy\Desktop\mobadra\csc_voda\combined_data_with_location.csv', index=False)


## convert all null in dim , taks approvel coulmn into N\A 

In [ ]:
# Fill NaN values in 'DIM' and 'Task' columns with 'N/A'
combined_df1[['DIM', 'Task', 'APPROVAL']] = combined_df1[['DIM', 'Task' , 'APPROVAL']].fillna('N/A')



## melt the data frame and remove the date rows from emp code

In [ ]:
# Reshape the date coulmn
combined_df = combined_df1.melt(id_vars=['Location','DIM', 'Task', 'APPROVAL'], var_name='Date', value_name='Value')





employee_codes = combined_df['DIM'].dropna().unique()
values_to_remove = ['Contracted Hours', 'Allocated Hours', 'Remaining', 'Technical',"Planning"]
df_employee_codes = pd.DataFrame(employee_codes, columns=['EmployeeCode'])
df_employee_codes_filtered = df_employee_codes[~df_employee_codes['EmployeeCode'].isin(values_to_remove)]

# Go through each row to update 'Date' column
for index, row in combined_df.iterrows():
    if row['DIM'] in df_employee_codes_filtered['EmployeeCode'].values:
        if pd.notna(row['Date']):
            combined_df.at[index, 'Date'] = ''

# Save the combined DataFrame to a new CSV file
# combined_df.to_csv(r'C:\Users\Hazem Hamdy\Desktop\mobadra\csc_voda\combined_data.csv', index=False)


## Extract the data of each employee and put it in another column,that Data assigned to it's emp_code

In [ ]:
# Initialize temp to None
temp = None

# Create new columns 'subdim' and 'dim1'
combined_df['Sub-Dim'] = ''
combined_df['Dim'] = ''

# Iterate through the rows of combined_df
for index, row in combined_df.iterrows():
    if row['DIM'] in df_employee_codes_filtered['EmployeeCode'].values:
        # If DIM value is an employee code, set temp
        temp = row['DIM']
    else:
        # If DIM value is not an employee code, set 'subdim' and 'dim1'
        combined_df.at[index, 'Sub-Dim'] = row['DIM']
        combined_df.at[index, 'Dim'] = temp


## remove the emp_code from the column dim first and after  drop the old Dim coulms

In [ ]:
# Check if each value in 'DIM' exists in 'df_employee_codes_filtered' and remove those rows
combined_df_sin_emp = combined_df[~combined_df['DIM'].isin(df_employee_codes_filtered['EmployeeCode'])]
combined_df_sin_emp = combined_df_sin_emp.drop(columns=['DIM'])[['Location', 'Dim', 'Sub-Dim', 'Task', 'APPROVAL', 'Date', 'Value']]
combined_df_final = combined_df_sin_emp.dropna(subset=['Sub-Dim'])
# combined_df_final.to_csv(r'C:\Users\Hazem Hamdy\Desktop\mobadra\csc_voda\combined_data_final.csv', index=False)
# combined_df_final.to_csv(r'C:\Users\Hazem Hamdy\Desktop\mobadra\csc_voda\Emp_data_State_final.csv', index=False)

## Start with the Emp DATA

In [ ]:
employee_dep = pd.read_csv('C:\\Users\\Hazem Hamdy\\Desktop\\mobadra\\voda_employee\\Employee Dept.csv')
employee_cost = pd.read_csv('C:\\Users\\Hazem Hamdy\\Desktop\\mobadra\\voda_employee\\Employee HR Cost.csv')

## Merge Employee Dept & Employee HR Cost files

In [ ]:
employee_cost_dep = pd.merge(employee_dep,employee_cost, on='Employee',how='outer')

In [ ]:
employee_cost_dep.to_csv(r'C:\\Users\\Hazem Hamdy\\Desktop\\mobadra\\voda_employee\\employee_cost_dep.csv', index=False)

## Merge the two prevous dataframe

In [ ]:
emp_agg = pd.merge(combined_df_final,employee_cost_dep, left_on='Dim' , right_on='Employee',how='inner')


In [ ]:
emp_agg = emp_agg.drop(columns=['Employee'])

## Preproccessing

In [ ]:
emp_agg.duplicated().sum()

In [ ]:
emp_agg_filterd= emp_agg.drop_duplicates()

In [ ]:
emp_agg_filterd.duplicated().sum()

In [ ]:
emp_agg_filterd.to_csv(r'C:\\Users\\Hazem Hamdy\\Desktop\\mobadra\\voda_employee\\employee_agg.csv', index=False)

## Analsis&vis

In [ ]:
# Load your data into a DataFrame (skip this step if you already have the DataFrame)
df = pd.read_csv('C:\\Users\\Hazem Hamdy\\Desktop\\mobadra\\voda_employee\\employee_agg_final_final.csv')


## 1. Task Distribution by Employee

In [ ]:
# Count the number of tasks per employee
task_counts = df['Dim'].value_counts()
# Get the top 10 employees by task count
top_10_task_counts = task_counts.head(10)

# Set the size of the plot
plt.figure(figsize=(12, 6))

# Plot the data for the top 10 employees
sns.barplot(x=top_10_task_counts.index, y=top_10_task_counts.values, palette="viridis")

# Add labels and title
plt.xlabel('Employee')
plt.ylabel('Number of Tasks')
plt.title('Task Distribution for Top 10 Employees')
plt.xticks(rotation=45)  # Rotate x labels for better readability

# Show the plot
plt.show()


## 9. Location-Based Task Analysis

In [ ]:
# Pivot the data for task counts per location and approval status
location_task_analysis = df.groupby(['Location', 'APPROVAL']).size().unstack()

# Plot the stacked bar chart
location_task_analysis.plot(kind='bar', stacked=True, figsize=(12, 8), color=['#4CAF50', '#FFC107'])
plt.title('Location-Based Task Analysis')
plt.xlabel('Location')
plt.ylabel('Number of Tasks')
plt.xticks(rotation=45)
plt.legend(title='Approval Status')
plt.show()


## 3. Approval partial Status Across Different Tasks

In [ ]:
# Filter the data for tasks with 'Partial' approval
partial_approval_data = df[df['APPROVAL'] == 'Partial']

# Count the number of partial approval statuses per task
partial_approval_status = partial_approval_data['Task'].value_counts().head(15)

# Plot the bar chart
plt.figure(figsize=(12, 8))
partial_approval_status.plot(kind='bar', color='green')
plt.title('Number of Partial Approval Tasks per Task')
plt.xlabel('Task')
plt.ylabel('Count')
plt.xticks(rotation=70)
plt.show()


In [ ]:
# 1. Filter the data for the specific task 'TASK_T0397'
task_data = partial_approval_data[partial_approval_data['Task'] == 'TASK_T0397']

# 2. Count the number of times each employee has been assigned to this task
employee_counts = task_data['Dim'].value_counts()

# Plot the bar chart
plt.figure(figsize=(10, 6))
employee_counts.plot(kind='bar', color='skyblue')
plt.title('Number of Times Each Employee is Assigned to TASK_T0397')
plt.xlabel('Employee Code')
plt.ylabel('Number of Assignments')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()  # Adjust layout to fit labels
plt.show()


## 6. Task partial Rate by Employee

In [ ]:
# Filter the data for tasks with 'Partial' approval
partial_approval_data = df[df['APPROVAL'] == 'Partial']

# Count the number of partial approval statuses per task
partial_approval_status = partial_approval_data['Dim'].value_counts().head(15)

# Plot the bar chart
plt.figure(figsize=(12, 8))
partial_approval_status.plot(kind='bar', color='purple')
plt.title('Number of Partial Approval Tasks per Task')
plt.xlabel('Task')
plt.ylabel('Count')
plt.xticks(rotation=70)
plt.show()

## 8. Sub-Dimension vs. Value Analysis

In [ ]:
# Calculate average cost per Sub-Dim
avg_cost_per_subdim = df.groupby('Sub-Dim')['Value'].mean()

# Plot the bar chart
plt.figure(figsize=(12, 6))
avg_cost_per_subdim.plot(kind='bar', color='orange')
plt.title('Average Value per Sub-Dimension')
plt.xlabel('Sub-Dimension')
plt.ylabel('Average Value')
plt.xticks(rotation=45)
plt.show()


## pie char to each level

In [ ]:
# Count occurrences of each unique value in LEVEL-01
level_03_counts = df['LEVEL-03'].value_counts()

# Plot pie chart
plt.figure(figsize=(8, 6))
plt.pie(level_03_counts, labels=level_03_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('LEVEL-03 Distribution')
plt.show()


In [ ]:
# Count occurrences of each unique value in LEVEL-01
level_02_counts = df['LEVEL-02'].value_counts()

# Plot pie chart
plt.figure(figsize=(8, 6))
plt.pie(level_02_counts, labels=level_02_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('LEVEL-02 Distribution')
plt.show()


In [ ]:
# Count occurrences of each unique value in LEVEL-01
level_01_counts = df['LEVEL-01'].value_counts()

# Plot pie chart
plt.figure(figsize=(8, 6))
plt.pie(level_01_counts, labels=level_01_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('LEVEL-01 Distribution')
plt.show()


## 4. Monthly Value Trends 

In [ ]:
# Correct date format and parse the 'Date' column
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b')


# Aggregating total value by month
monthly_value = df.groupby(df['Date'].dt.to_period('M'))['Value'].sum()

# Plotting
plt.figure(figsize=(10, 6))
monthly_value.plot(kind='line', marker='o')
plt.title('Monthly Value Distribution')
plt.xlabel('Month')
plt.ylabel('Total Value')
plt.grid(True)
plt.show()
